In [30]:
import pandas as pd
import numpy as np
import altair as alt

from datapane import Plot, Table, Report
from datapane.api import Markdown

query = params.get('query', "Senator")
display_mode = params.get('display_mode', "View Data")
max_score = params.get('max_votes', 10000)
min_score = params.get('min_votes', 0)

search_results = pd.read_csv('https://data.hawaii.gov/api/views/eze4-hq7j/rows.csv?accessType=DOWNLOAD')
results_f = search_results[search_results['Election_Votes'].between(min_score, max_score)].dropna(subset=['Choice_party'])
results_f = results_f[results_f['Contest_title'].str.contains(query)]

if len(results_f) == 0:
    report = [Markdown(f"## Results\n Didn't find any relevant results for {query} :(")]
else:
    heading = Markdown(f"## Results\n Congratulations! You have {len(results_f)} search results.")
    
    alt.data_transformers.disable_max_rows()
    plot_base = alt.Chart(results_f).mark_area(
        opacity=0.3,
        interpolate='step'
    ).interactive()
    
    if display_mode == "View Data":
        report = [heading, Table.create(results_f)]
    elif display_mode == "Election Votes Plot":
        plot = plot_base.encode(
            alt.X('Election_Votes:Q', bin=alt.Bin(maxbins=10)),
            alt.Y('count()', stack=None),
            alt.Color('Choice_party')
        )
        report = [heading, Plot.create(plot)]
    elif display_mode == "Absentee Votes Plot":
        plot = plot_base.encode(
            alt.X('Absentee_votes:Q', bin=alt.Bin(maxbins=10)),
            alt.Y('count()', stack=None),
            alt.Color('Choice_party')
        )
        report = [heading, Plot.create(plot)]
    
# Report.create(*report).preview()

18:17:13 [INFO ] Uploading assets for Report
18:17:13 [INFO ] Uploading /var/folders/_x/188lgf3s507dygvl0b1l8rn40000gn/T/datapane-temp-lnavwdvi.json.gz
